Train one MLP for each slice. Average the predictions of the 11 MLPs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from pathlib import Path
import torch.nn.functional as F
import os
import gc
from collections import defaultdict

# --- Configuration (Should match training script) ---
FEATURE_ROOT = Path("/data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130")
CSV_PATH = Path("/data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv")
MODELS_OUTPUT_DIR = Path("./slice_specific_models") # Directory where best models are saved
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32 # Use the same batch size as validation/training if possible, or adjust
NUM_WORKERS = 4

# Define the target coronal slices (Must match training and preprocessing)
CORONAL_SLICE_CENTER = 125
CORONAL_SLICE_RANGE = 5
TARGET_CORONAL_INDICES = list(range(CORONAL_SLICE_CENTER - CORONAL_SLICE_RANGE,
                                    CORONAL_SLICE_CENTER + CORONAL_SLICE_RANGE + 1)) # e.g., 120 to 130

# --- Model Definition (Ensure this class definition is available) ---
class AgeMLPWithAttentionBN(nn.Module):
    # ... (Paste the full class definition here as in the training script) ...
    def __init__(self, input_dim=256, embed_dim=256, num_heads=8,
                 hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, dropout_rate=0.3):
        super(AgeMLPWithAttentionBN, self).__init__()
        if embed_dim % num_heads != 0:
            raise ValueError(f"embed_dim ({embed_dim}) must be divisible by num_heads ({num_heads})")
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=False)
        self.norm_attn = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1), nn.BatchNorm1d(hidden_dim1), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim1, hidden_dim2), nn.BatchNorm1d(hidden_dim2), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim2, hidden_dim3), nn.BatchNorm1d(hidden_dim3), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim3, 1)
        )

    def forward(self, x):
        x_attn = x.unsqueeze(0)
        attn_output, _ = self.attention(x_attn, x_attn, x_attn)
        x = self.norm_attn(x_attn + attn_output)
        x = x.squeeze(0)
        output = self.mlp(x)
        return output.squeeze(-1)


# --- Dataset Definition (Modified to return subject ID) ---
class SliceAgePredictionDataset(Dataset):
    # ... (Paste the full class definition here, but modify __getitem__) ...
    def __init__(self, feature_root, csv_path, split, slice_index):
        self.feature_root = Path(feature_root)
        self.csv_path = Path(csv_path)
        self.split = split
        self.slice_index = slice_index
        self.split_dir = self.feature_root / self.split

        if not self.split_dir.is_dir():
            raise FileNotFoundError(f"Split directory not found: {self.split_dir}")
        if not self.csv_path.is_file():
            raise FileNotFoundError(f"CSV file not found: {self.csv_path}")

        try:
            df = pd.read_csv(self.csv_path)
            self.meta_dict = df.set_index('filename')['age'].to_dict()
            self.subject_id_to_filename = {
                fname.replace(".nii.gz", "").replace(".nii", ""): fname
                for fname in self.meta_dict.keys()
            }
        except Exception as e:
            raise ValueError(f"Error loading or processing CSV {self.csv_path}: {e}")

        all_subject_dirs = [d for d in self.split_dir.iterdir() if d.is_dir()]
        self.valid_slice_files = []
        self.file_to_subject_id = {} # Map file path back to subject_id

        for subject_dir in all_subject_dirs:
            subject_id = subject_dir.name
            slice_filename = f"slice_{self.slice_index}.npy"
            expected_slice_path = subject_dir / slice_filename
            original_filename = self.subject_id_to_filename.get(subject_id)

            if original_filename and original_filename in self.meta_dict:
                if expected_slice_path.is_file():
                    self.valid_slice_files.append(expected_slice_path)
                    self.file_to_subject_id[expected_slice_path] = subject_id # Store mapping

        if not self.valid_slice_files:
             raise RuntimeError(f"No valid slice files found for slice {self.slice_index} in {self.split_dir} with matching metadata.")
        print(f"Found {len(self.valid_slice_files)} valid files for slice {self.slice_index} in split {self.split}.")


    def __len__(self):
        return len(self.valid_slice_files)

    def __getitem__(self, idx):
        slice_path = self.valid_slice_files[idx]
        subject_id = self.file_to_subject_id[slice_path] # Retrieve subject_id
        original_filename = self.subject_id_to_filename.get(subject_id)

        if not original_filename:
             raise ValueError(f"Could not find original filename for subject ID {subject_id}")

        try:
            embedding = np.load(slice_path)
            embedding_tensor = torch.tensor(embedding, dtype=torch.float32)

            if embedding_tensor.shape == (1, 256, 64, 64):
                 pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor, (1, 1)).squeeze()
            elif embedding_tensor.shape == (256, 64, 64):
                 pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor.unsqueeze(0), (1, 1)).squeeze()
            elif embedding_tensor.shape == (256,):
                 pooled_embedding = embedding_tensor
            else:
                 raise ValueError(f"Unexpected embedding shape {embedding_tensor.shape} for {slice_path}")

            if pooled_embedding.shape[0] != 256:
                 raise ValueError(f"Pooled embedding shape is not 256 for {slice_path}: {pooled_embedding.shape}")

            age = self.meta_dict[original_filename]
            age_tensor = torch.tensor(age, dtype=torch.float32)

            # Return subject_id along with features and age
            return pooled_embedding, age_tensor, subject_id

        except Exception as e:
            print(f"Error loading or processing file {slice_path}: {e}")
            raise e

# --- Evaluation Function (Modified to return predictions and IDs) ---
@torch.no_grad()
def predict_on_test_set(model, loader, device):
    model.eval()
    all_predictions = []
    all_ages = []
    all_subject_ids = []

    for features, ages, subject_ids in loader:
        features = features.to(device)
        # ages = ages.to(device) # No need to move ages if only used on CPU later

        predictions = model(features)

        all_predictions.extend(predictions.cpu().numpy())
        all_ages.extend(ages.cpu().numpy())
        all_subject_ids.extend(subject_ids) # subject_ids are already strings

    return np.array(all_predictions), np.array(all_ages), all_subject_ids


# --- Main Testing Logic ---
print(f"Starting testing on device: {DEVICE}")
print(f"Loading models from: {MODELS_OUTPUT_DIR}")
print(f"Testing on slices: {TARGET_CORONAL_INDICES}")

individual_model_maes = {}
# Use defaultdict to easily append predictions per subject
# Stores {subject_id: [pred_slice_1, pred_slice_2, ...]}
all_slice_predictions = defaultdict(list)
# Stores {subject_id: ground_truth_age} - only need to store once
ground_truth_ages = {}

for slice_idx in TARGET_CORONAL_INDICES:
    print(f"\n--- Testing Slice: {slice_idx} ---")
    model_path = MODELS_OUTPUT_DIR / f"best_model_slice_{slice_idx}.pth"

    if not model_path.exists():
        print(f"Warning: Model file not found for slice {slice_idx} at {model_path}. Skipping.")
        individual_model_maes[slice_idx] = np.nan
        continue

    # --- Load Data for the current slice ---
    try:
        test_dataset = SliceAgePredictionDataset(FEATURE_ROOT, CSV_PATH, 'test', slice_idx)
        # Important: Use shuffle=False for testing to keep order consistent if needed,
        # and to correctly map predictions back to subjects.
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                                 num_workers=NUM_WORKERS, pin_memory=True)
    except (FileNotFoundError, ValueError, RuntimeError) as e:
        print(f"Error initializing dataset/loader for test split, slice {slice_idx}: {e}")
        print(f"Skipping testing for slice {slice_idx}.")
        individual_model_maes[slice_idx] = np.nan
        continue

    # --- Load Model ---
    model = AgeMLPWithAttentionBN().to(DEVICE)
    try:
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        print(f"Loaded model weights from {model_path}")
    except Exception as e:
        print(f"Error loading model weights for slice {slice_idx}: {e}")
        individual_model_maes[slice_idx] = np.nan
        del model, test_dataset, test_loader
        gc.collect()
        if DEVICE.startswith('cuda'): torch.cuda.empty_cache()
        continue

    # --- Get Predictions ---
    predictions, ages, subject_ids = predict_on_test_set(model, test_loader, DEVICE)

    # --- Calculate and Store Individual MAE ---
    if len(predictions) > 0:
        mae = np.mean(np.abs(predictions - ages))
        individual_model_maes[slice_idx] = mae
        print(f"Slice {slice_idx} Test MAE: {mae:.3f}")

        # --- Store predictions and ground truth for averaging ---
        for i, subj_id in enumerate(subject_ids):
            all_slice_predictions[subj_id].append(predictions[i])
            # Store ground truth age only once per subject
            if subj_id not in ground_truth_ages:
                ground_truth_ages[subj_id] = ages[i]
    else:
        print(f"No predictions generated for slice {slice_idx}.")
        individual_model_maes[slice_idx] = np.nan


    # Clean up GPU memory
    del model, test_dataset, test_loader, predictions, ages, subject_ids
    gc.collect()
    if DEVICE.startswith('cuda'):
        torch.cuda.empty_cache()


# --- Calculate MAE for Averaged Predictions ---
print("\n--- Calculating MAE for Averaged Predictions ---")
average_predictions = []
corresponding_ground_truths = []
subjects_with_complete_preds = 0

# Check subjects that have predictions for *all* target slices
num_target_slices = len(TARGET_CORONAL_INDICES)
subject_ids_in_order = sorted(all_slice_predictions.keys()) # Ensure consistent order

for subj_id in subject_ids_in_order:
    preds = all_slice_predictions[subj_id]
    if len(preds) == num_target_slices: # Ensure we have a prediction from each slice model
        avg_pred = np.mean(preds)
        average_predictions.append(avg_pred)
        corresponding_ground_truths.append(ground_truth_ages[subj_id])
        subjects_with_complete_preds += 1
    else:
        print(f"Warning: Subject {subj_id} has {len(preds)} predictions, expected {num_target_slices}. Skipping for average MAE calculation.")

if subjects_with_complete_preds > 0:
    average_predictions = np.array(average_predictions)
    corresponding_ground_truths = np.array(corresponding_ground_truths)
    average_mae = np.mean(np.abs(average_predictions - corresponding_ground_truths))
    print(f"\nNumber of subjects with complete predictions across all {num_target_slices} slices: {subjects_with_complete_preds}")
    print(f"MAE using averaged predictions: {average_mae:.3f}")
else:
    print("\nCould not calculate average MAE: No subjects had predictions for all required slices.")


# --- Final Summary ---
print("\n--- Individual Slice Model Test MAEs ---")
for slice_idx in TARGET_CORONAL_INDICES:
    mae = individual_model_maes.get(slice_idx, 'N/A')
    if isinstance(mae, float) and not np.isnan(mae):
        print(f"Slice {slice_idx}: {mae:.3f}")
    else:
        print(f"Slice {slice_idx}: {mae}")


Starting testing on device: cuda:1
Loading models from: slice_specific_models
Testing on slices: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]

--- Testing Slice: 120 ---
Found 296 valid files for slice 120 in split test.
Loaded model weights from slice_specific_models/best_model_slice_120.pth
Slice 120 Test MAE: 7.078

--- Testing Slice: 121 ---
Found 296 valid files for slice 121 in split test.
Loaded model weights from slice_specific_models/best_model_slice_121.pth
Slice 121 Test MAE: 7.898

--- Testing Slice: 122 ---
Found 296 valid files for slice 122 in split test.
Loaded model weights from slice_specific_models/best_model_slice_122.pth
Slice 122 Test MAE: 7.434

--- Testing Slice: 123 ---
Found 296 valid files for slice 123 in split test.
Loaded model weights from slice_specific_models/best_model_slice_123.pth
Slice 123 Test MAE: 7.943

--- Testing Slice: 124 ---
Found 296 valid files for slice 124 in split test.
Loaded model weights from slice_specific_models/best_mod

Early fusion with concatenation

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from pathlib import Path
import torch.nn.functional as F
import os
import gc
from tqdm import tqdm
import random

# --- Configuration ---
FEATURE_ROOT = Path("/data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130")
CSV_PATH = Path("/data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv")
MODEL_SAVE_PATH = Path("./concatenated_mlp_model.pth") # Path to save the best model
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
SPLITS = ["train", "validation", "test"]

# Define the target coronal slices (Must match preprocessing)
CORONAL_SLICE_CENTER = 125
CORONAL_SLICE_RANGE = 5
TARGET_CORONAL_INDICES = list(range(CORONAL_SLICE_CENTER - CORONAL_SLICE_RANGE,
                                    CORONAL_SLICE_CENTER + CORONAL_SLICE_RANGE + 1)) # e.g., 120 to 130
NUM_SLICES = len(TARGET_CORONAL_INDICES)
FEATURE_DIM_PER_SLICE = 256 # After Global Average Pooling
CONCAT_FEATURE_DIM = NUM_SLICES * FEATURE_DIM_PER_SLICE # e.g., 11 * 256 = 2816

# --- Hyperparameters ---
EPOCHS = 500
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.4 # Might need adjustment for the larger input dim
SCHEDULER_PATIENCE_PERCENT = 0.10
EARLY_STOPPING_PATIENCE = 50
NUM_WORKERS = 4

# --- Dataset Definition (Loads and Concatenates Features) ---
class ConcatenatedSliceDataset(Dataset):
    def __init__(self, feature_root, csv_path, split, slice_indices):
        """
        Args:
            feature_root (Path): Base directory containing split folders.
            csv_path (Path): Path to the CSV file with metadata.
            split (str): The dataset split ('train', 'validation', or 'test').
            slice_indices (list[int]): List of slice indices to load and concatenate.
        """
        self.feature_root = Path(feature_root)
        self.csv_path = Path(csv_path)
        self.split = split
        self.slice_indices = sorted(slice_indices) # Ensure consistent order
        self.num_slices = len(self.slice_indices)
        self.split_dir = self.feature_root / self.split

        print(f"\n[Dataset Init] Split: {self.split}, Concatenating Slices: {self.slice_indices}")
        print(f"Loading features from base: {self.feature_root}")
        print(f"Loading metadata from: {self.csv_path}")

        if not self.split_dir.is_dir():
            raise FileNotFoundError(f"Split directory not found: {self.split_dir}")
        if not self.csv_path.is_file():
            raise FileNotFoundError(f"CSV file not found: {self.csv_path}")

        try:
            df = pd.read_csv(self.csv_path)
            self.meta_dict = df.set_index('filename')['age'].to_dict()
            self.subject_id_to_filename = {
                fname.replace(".nii.gz", "").replace(".nii", ""): fname
                for fname in self.meta_dict.keys()
            }
            print(f"Loaded metadata for {len(self.meta_dict)} subjects from CSV.")
        except Exception as e:
            raise ValueError(f"Error loading or processing CSV {self.csv_path}: {e}")

        # Find subject directories that have metadata and *all* required slice files
        all_subject_dirs = [d for d in self.split_dir.iterdir() if d.is_dir()]
        self.valid_subject_dirs = []
        missing_meta_count = 0
        incomplete_slice_count = 0

        print(f"Scanning {len(all_subject_dirs)} potential subject directories...")

        for subject_dir in all_subject_dirs:
            subject_id = subject_dir.name
            original_filename = self.subject_id_to_filename.get(subject_id)

            if original_filename and original_filename in self.meta_dict:
                # Check if all required slice files exist for this subject
                all_slices_present = True
                for slice_idx in self.slice_indices:
                    slice_filename = f"slice_{slice_idx}.npy"
                    expected_slice_path = subject_dir / slice_filename
                    if not expected_slice_path.is_file():
                        all_slices_present = False
                        break # No need to check further slices for this subject

                if all_slices_present:
                    self.valid_subject_dirs.append(subject_dir)
                else:
                    incomplete_slice_count += 1
            else:
                missing_meta_count += 1

        if missing_meta_count > 0:
            print(f"Warning: {missing_meta_count} subject directories did not have corresponding metadata.")
        if incomplete_slice_count > 0:
            print(f"Warning: {incomplete_slice_count} subjects with metadata were missing one or more required slices.")

        if not self.valid_subject_dirs:
             raise RuntimeError(f"No subjects found in {self.split_dir} with metadata and all required slices {self.slice_indices}.")

        print(f"Found {len(self.valid_subject_dirs)} valid subjects for split {self.split}.")

    def __len__(self):
        return len(self.valid_subject_dirs)

    def __getitem__(self, idx):
        subject_dir = self.valid_subject_dirs[idx]
        subject_id = subject_dir.name
        original_filename = self.subject_id_to_filename.get(subject_id)

        if not original_filename:
             raise ValueError(f"Could not find original filename for subject ID {subject_id}")

        concatenated_features = []
        try:
            for slice_idx in self.slice_indices:
                slice_filename = f"slice_{slice_idx}.npy"
                slice_path = subject_dir / slice_filename
                embedding = np.load(slice_path)
                embedding_tensor = torch.tensor(embedding, dtype=torch.float32)

                # Apply Global Average Pooling (GAP)
                if embedding_tensor.shape == (1, 256, 64, 64):
                    pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor, (1, 1)).squeeze() # -> (256,)
                elif embedding_tensor.shape == (256, 64, 64):
                    pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor.unsqueeze(0), (1, 1)).squeeze()
                elif embedding_tensor.shape == (256,):
                    pooled_embedding = embedding_tensor
                else:
                    raise ValueError(f"Unexpected embedding shape {embedding_tensor.shape} for {slice_path}")

                if pooled_embedding.shape[0] != FEATURE_DIM_PER_SLICE:
                    raise ValueError(f"Pooled embedding shape is not {FEATURE_DIM_PER_SLICE} for {slice_path}: {pooled_embedding.shape}")

                concatenated_features.append(pooled_embedding)

            # Concatenate features from all slices for this subject
            final_feature_vector = torch.cat(concatenated_features, dim=0) # Shape: (num_slices * 256,)

            # Get the age
            age = self.meta_dict[original_filename]
            age_tensor = torch.tensor(age, dtype=torch.float32)

            return final_feature_vector, age_tensor

        except Exception as e:
            print(f"Error loading or processing slices for subject {subject_dir}: {e}")
            raise e

# --- Model Definition (Takes concatenated features as input) ---
class ConcatenatedAgeMLP(nn.Module):
    def __init__(self, input_dim=CONCAT_FEATURE_DIM, hidden_dim1=1024, hidden_dim2=512, hidden_dim3=256, dropout_rate=DROPOUT_RATE):
        """
        MLP for age prediction using concatenated features from multiple slices.
        Args:
            input_dim (int): Dimension of the concatenated input feature vector (num_slices * feature_dim_per_slice).
            hidden_dim1/2/3 (int): Sizes of the hidden layers.
            dropout_rate (float): Dropout probability.
        """
        super(ConcatenatedAgeMLP, self).__init__()
        self.mlp = nn.Sequential(
            # Layer 1
            nn.Linear(input_dim, hidden_dim1),
            nn.BatchNorm1d(hidden_dim1), # Batch Norm is often helpful
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # Layer 2
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.BatchNorm1d(hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # Layer 3
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.BatchNorm1d(hidden_dim3),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # Output Layer
            nn.Linear(hidden_dim3, 1) # Output is a single value (age)
        )

    def forward(self, x):
        # x shape: (batch_size, input_dim)
        output = self.mlp(x)
        return output.squeeze(-1) # (batch_size,)


# --- Training and Evaluation Functions (Unchanged from previous script) ---
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    num_samples = 0
    for features, ages in loader:
        features, ages = features.to(device), ages.to(device)
        optimizer.zero_grad()
        predictions = model(features)
        loss = criterion(predictions, ages)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * features.size(0)
        num_samples += features.size(0)
    return total_loss / num_samples

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_mae = 0.0
    num_samples = 0
    for features, ages in loader:
        features, ages = features.to(device), ages.to(device)
        predictions = model(features)
        loss = criterion(predictions, ages) # Use the same criterion (e.g., L1Loss)
        total_loss += loss.item() * features.size(0)
        mae = F.l1_loss(predictions, ages, reduction='sum') # Sum MAE for batch
        total_mae += mae.item()
        num_samples += features.size(0)

    avg_loss = total_loss / num_samples
    avg_mae = total_mae / num_samples
    return avg_loss, avg_mae

# --- Main Training Script ---
if __name__ == "__main__":
    print(f"Starting concatenated feature MLP training...")
    print(f"Target Slices: {TARGET_CORONAL_INDICES}")
    print(f"Concatenated Feature Dim: {CONCAT_FEATURE_DIM}")
    print(f"Using device: {DEVICE}")
    print(f"Feature Root: {FEATURE_ROOT}")
    print(f"CSV Path: {CSV_PATH}")
    print(f"Best model will be saved to: {MODEL_SAVE_PATH}")
    print(f"Hyperparameters: Epochs={EPOCHS}, LR={LEARNING_RATE}, Batch={BATCH_SIZE}, Dropout={DROPOUT_RATE}, ES_Patience={EARLY_STOPPING_PATIENCE}")

    # --- Setup Datasets and DataLoaders ---
    try:
        print("Setting up datasets...")
        train_dataset = ConcatenatedSliceDataset(FEATURE_ROOT, CSV_PATH, 'train', TARGET_CORONAL_INDICES)
        val_dataset = ConcatenatedSliceDataset(FEATURE_ROOT, CSV_PATH, 'validation', TARGET_CORONAL_INDICES)
        # Optional: test_dataset = ConcatenatedSliceDataset(FEATURE_ROOT, CSV_PATH, 'test', TARGET_CORONAL_INDICES)

        print("Setting up dataloaders...")
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
        # Optional: test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    except (FileNotFoundError, ValueError, RuntimeError) as e:
        print(f"Error initializing datasets/loaders: {e}")
        raise SystemExit("Dataset/Loader initialization failed.")

    # --- Setup Model, Loss, Optimizer, Scheduler ---
    print("Initializing model, optimizer, scheduler...")
    model = ConcatenatedAgeMLP(input_dim=CONCAT_FEATURE_DIM, dropout_rate=DROPOUT_RATE).to(DEVICE)
    criterion = nn.L1Loss() # MAE Loss
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    scheduler_patience_epochs = max(1, int(EPOCHS * SCHEDULER_PATIENCE_PERCENT))
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=scheduler_patience_epochs, factor=0.5, verbose=True)

    # --- Training Loop ---
    best_val_mae = float('inf')
    epochs_no_improve = 0
    best_epoch = -1
    history = {'train_loss': [], 'val_loss': [], 'val_mae': []}

    print(f"\n--- Starting Training ---")
    for epoch in range(EPOCHS):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
        val_loss, val_mae = evaluate(model, val_loader, criterion, DEVICE)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_mae'].append(val_mae)

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.3f}")

        scheduler.step(val_loss) # Step scheduler based on validation loss

        # --- Early Stopping & Model Saving ---
        if val_mae < best_val_mae:
            best_val_mae = val_mae
            best_epoch = epoch + 1
            epochs_no_improve = 0
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"  -> New best Val MAE: {best_val_mae:.3f}. Saved model to {MODEL_SAVE_PATH}")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
            print(f"\nEarly stopping triggered after {EARLY_STOPPING_PATIENCE} epochs without improvement.")
            break

        # Optional: Clean up GPU memory periodically
        if (epoch + 1) % 20 == 0:
             gc.collect()
             if DEVICE.startswith('cuda'):
                 torch.cuda.empty_cache()

    print(f"\n--- Training Finished ---")
    if best_epoch != -1:
         print(f"Best Validation MAE: {best_val_mae:.3f} achieved at epoch {best_epoch}")
         print(f"Best model saved to {MODEL_SAVE_PATH}")
    else:
         print("No improvement found during training. Model not saved.")

    # --- Optional: Evaluate on Test Set ---
    # if 'test_loader' in locals() and MODEL_SAVE_PATH.exists():
    #     print(f"\n--- Evaluating on Test Set using Best Model ---")
    #     try:
    #         model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    #         test_loss, test_mae = evaluate(model, test_loader, criterion, DEVICE)
    #         print(f"Test Loss: {test_loss:.4f} | Test MAE: {test_mae:.3f}")
    #     except Exception as e:
    #         print(f"Error during test set evaluation: {e}")
    # else:
    #     print("\nSkipping test set evaluation.")

    # You can plot history['train_loss'], history['val_loss'], history['val_mae'] here if needed
    # import matplotlib.pyplot as plt
    # plt.figure(...) etc.

    print("\nConcatenated MLP training script finished.")

Starting concatenated feature MLP training...
Target Slices: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
Concatenated Feature Dim: 2816
Using device: cuda:1
Feature Root: /data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130
CSV Path: /data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv
Best model will be saved to: concatenated_mlp_model.pth
Hyperparameters: Epochs=500, LR=0.0001, Batch=32, Dropout=0.4, ES_Patience=50
Setting up datasets...

[Dataset Init] Split: train, Concatenating Slices: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
Loading features from base: /data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130
Loading metadata from: /data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv
Loaded metadata for 2850 subjects from CSV.
Scanning 2275 potential subject directories...
Found 2274 valid subjects for split train.

[Dataset Init] Split: validation, Concatenating Slices: [120, 121, 1

Evaluating early fusion MLP

In [4]:
@torch.no_grad()
def test_model(model, test_loader, device):
    """
    Evaluate the trained model on the test dataset.

    Args:
        model (nn.Module): Trained model.
        test_loader (DataLoader): DataLoader for the test dataset.
        device (str): Device to run the evaluation on ('cpu' or 'cuda').

    Returns:
        float: Mean Absolute Error (MAE) on the test dataset.
    """
    model.eval()
    total_mae = 0.0
    num_samples = 0

    for features, ages in test_loader:
        features, ages = features.to(device), ages.to(device)
        predictions = model(features)
        mae = F.l1_loss(predictions, ages, reduction='sum')  # Sum MAE for the batch
        total_mae += mae.item()
        num_samples += features.size(0)

    avg_mae = total_mae / num_samples
    print(f"Test MAE: {avg_mae:.3f}")
    return avg_mae

# Example usage:
# Ensure the model is loaded with the best weights
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
test_dataset = ConcatenatedSliceDataset(FEATURE_ROOT, CSV_PATH, 'test', TARGET_CORONAL_INDICES)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# Evaluate the model on the test set
test_mae = test_model(model, test_loader, DEVICE)


[Dataset Init] Split: test, Concatenating Slices: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
Loading features from base: /data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130
Loading metadata from: /data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv
Loaded metadata for 2850 subjects from CSV.
Scanning 296 potential subject directories...
Found 296 valid subjects for split test.
Test MAE: 6.348


Feature Fusion with Multi-head Attention

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from pathlib import Path
import torch.nn.functional as F
import os
import gc
from tqdm import tqdm
import random

# --- Configuration ---
# IMPORTANT: Point to the directory containing the coronal 120-130 slice features
FEATURE_ROOT = Path("/data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130")
CSV_PATH = Path("/data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv")
MODEL_SAVE_PATH = Path("./fused_mlp_model_avg_pool_coronal_120_130.pth") # Path to save the best model
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
SPLITS = ["train", "validation", "test"]

# Define the target coronal slices to load and fuse
CORONAL_SLICE_CENTER = 125
CORONAL_SLICE_RANGE = 5
TARGET_CORONAL_INDICES = list(range(CORONAL_SLICE_CENTER - CORONAL_SLICE_RANGE,
                                    CORONAL_SLICE_CENTER + CORONAL_SLICE_RANGE + 1)) # 120 to 130
# Convert indices to strings to match the dataset logic expecting identifiers
TARGET_SLICE_IDENTIFIERS = [str(idx) for idx in TARGET_CORONAL_INDICES]

NUM_SLICES_TO_FUSE = len(TARGET_SLICE_IDENTIFIERS)
FEATURE_DIM_PER_SLICE = 256 # Dimension after Global Average Pooling in preprocessing

# --- Hyperparameters ---
EPOCHS = 500
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3 # Start similar to single-slice models, may need tuning
SCHEDULER_PATIENCE_PERCENT = 0.10
EARLY_STOPPING_PATIENCE = 40
NUM_WORKERS = 4
FUSION_METHOD = 'average' # 'average' or 'max'

# --- Dataset Definition (Loads and Fuses Features) ---
class FusedSliceDataset(Dataset):
    def __init__(self, feature_root, csv_path, split, target_slice_identifiers):
        """
        Args:
            feature_root (Path): Base directory containing split folders with subject subdirs.
            csv_path (Path): Path to the CSV file with metadata.
            split (str): The dataset split ('train', 'validation', or 'test').
            target_slice_identifiers (list[str]): List of slice identifiers (e.g., '120', '121').
        """
        self.feature_root = Path(feature_root)
        self.csv_path = Path(csv_path)
        self.split = split
        self.target_slice_identifiers = sorted(target_slice_identifiers) # Ensure consistent order
        self.num_slices_to_fuse = len(self.target_slice_identifiers)
        self.split_dir = self.feature_root / self.split

        print(f"\n[Dataset Init] Split: {self.split}, Fusing Coronal Slices: {self.target_slice_identifiers}")
        print(f"Loading features from base: {self.feature_root}")
        print(f"Loading metadata from: {self.csv_path}")

        if not self.split_dir.is_dir():
            raise FileNotFoundError(f"Split directory not found: {self.split_dir}")
        if not self.csv_path.is_file():
            raise FileNotFoundError(f"CSV file not found: {self.csv_path}")

        try:
            df = pd.read_csv(self.csv_path)
            self.meta_dict = df.set_index('filename')['age'].to_dict()
            self.subject_id_to_filename = {
                fname.replace(".nii.gz", "").replace(".nii", ""): fname
                for fname in self.meta_dict.keys()
            }
            print(f"Loaded metadata for {len(self.meta_dict)} subjects from CSV.")
        except Exception as e:
            raise ValueError(f"Error loading or processing CSV {self.csv_path}: {e}")

        all_subject_dirs = [d for d in self.split_dir.iterdir() if d.is_dir()]
        self.valid_subject_dirs = []
        missing_meta_count = 0
        incomplete_slice_count = 0

        print(f"Scanning {len(all_subject_dirs)} potential subject directories...")

        for subject_dir in all_subject_dirs:
            subject_id = subject_dir.name
            original_filename = self.subject_id_to_filename.get(subject_id)

            if original_filename and original_filename in self.meta_dict:
                all_slices_present = True
                for slice_id in self.target_slice_identifiers:
                    # Construct filename based on the coronal slice index
                    slice_filename = f"slice_{slice_id}.npy"
                    expected_slice_path = subject_dir / slice_filename
                    if not expected_slice_path.is_file():
                        all_slices_present = False
                        break

                if all_slices_present:
                    self.valid_subject_dirs.append(subject_dir)
                else:
                    incomplete_slice_count += 1
            else:
                missing_meta_count += 1

        if missing_meta_count > 0:
            print(f"Warning: {missing_meta_count} subject directories did not have corresponding metadata.")
        if incomplete_slice_count > 0:
            print(f"Warning: {incomplete_slice_count} subjects with metadata were missing one or more required slices.")

        if not self.valid_subject_dirs:
             raise RuntimeError(f"No subjects found in {self.split_dir} with metadata and all required slices {self.target_slice_identifiers}.")

        print(f"Found {len(self.valid_subject_dirs)} valid subjects for split {self.split}.")

    def __len__(self):
        return len(self.valid_subject_dirs)

    def __getitem__(self, idx):
        subject_dir = self.valid_subject_dirs[idx]
        subject_id = subject_dir.name
        original_filename = self.subject_id_to_filename.get(subject_id)

        if not original_filename:
             raise ValueError(f"Could not find original filename for subject ID {subject_id}")

        slice_features = []
        try:
            for slice_id in self.target_slice_identifiers:
                slice_filename = f"slice_{slice_id}.npy"
                slice_path = subject_dir / slice_filename
                embedding = np.load(slice_path)
                embedding_tensor = torch.tensor(embedding, dtype=torch.float32)

                # Apply Global Average Pooling (GAP) if not already done
                if embedding_tensor.ndim == 4 and embedding_tensor.shape[0] == 1 and embedding_tensor.shape[1] == FEATURE_DIM_PER_SLICE:
                    pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor, (1, 1)).squeeze()
                elif embedding_tensor.ndim == 3 and embedding_tensor.shape[0] == FEATURE_DIM_PER_SLICE:
                     pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor.unsqueeze(0), (1, 1)).squeeze()
                elif embedding_tensor.ndim == 1 and embedding_tensor.shape[0] == FEATURE_DIM_PER_SLICE:
                     pooled_embedding = embedding_tensor
                else:
                    raise ValueError(f"Unexpected embedding shape {embedding_tensor.shape} for {slice_path}.")

                if pooled_embedding.shape[0] != FEATURE_DIM_PER_SLICE:
                    raise ValueError(f"Pooled embedding shape is not {FEATURE_DIM_PER_SLICE} for {slice_path}: {pooled_embedding.shape}")

                slice_features.append(pooled_embedding)

            stacked_features = torch.stack(slice_features, dim=0)

            if FUSION_METHOD == 'average':
                fused_feature_vector = torch.mean(stacked_features, dim=0)
            # elif FUSION_METHOD == 'max':
            #     fused_feature_vector = torch.max(stacked_features, dim=0)[0]
            else:
                raise ValueError(f"Unknown fusion method: {FUSION_METHOD}")

            age = self.meta_dict[original_filename]
            age_tensor = torch.tensor(age, dtype=torch.float32)

            return fused_feature_vector, age_tensor

        except Exception as e:
            print(f"Error loading or processing slices for subject {subject_dir}: {e}")
            raise e

# --- Model Definition (Takes FUSED features as input) ---
# Input dim is FEATURE_DIM_PER_SLICE (256 after fusion)
class FusedAgeMLP(nn.Module):
    def __init__(self, input_dim=FEATURE_DIM_PER_SLICE, hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, dropout_rate=DROPOUT_RATE):
        super(FusedAgeMLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1), nn.BatchNorm1d(hidden_dim1), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim1, hidden_dim2), nn.BatchNorm1d(hidden_dim2), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim2, hidden_dim3), nn.BatchNorm1d(hidden_dim3), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim3, 1)
        )

    def forward(self, x):
        return self.mlp(x).squeeze(-1)

# --- Training and Evaluation Functions (Identical) ---
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    num_samples = 0
    pbar = tqdm(loader, desc="Training", leave=False)
    for features, ages in pbar:
        features, ages = features.to(device), ages.to(device)
        optimizer.zero_grad()
        predictions = model(features)
        loss = criterion(predictions, ages)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * features.size(0)
        num_samples += features.size(0)
        pbar.set_postfix(loss=loss.item())
    return total_loss / num_samples

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_mae = 0.0
    num_samples = 0
    pbar = tqdm(loader, desc="Evaluating", leave=False)
    for features, ages in pbar:
        features, ages = features.to(device), ages.to(device)
        predictions = model(features)
        loss = criterion(predictions, ages)
        total_loss += loss.item() * features.size(0)
        mae = F.l1_loss(predictions, ages, reduction='sum')
        total_mae += mae.item()
        num_samples += features.size(0)
        pbar.set_postfix(loss=loss.item(), mae=(total_mae/num_samples if num_samples > 0 else 0))

    avg_loss = total_loss / num_samples if num_samples > 0 else 0
    avg_mae = total_mae / num_samples if num_samples > 0 else 0
    return avg_loss, avg_mae

# --- Main Training Script ---
if __name__ == "__main__":
    print(f"Starting feature fusion ({FUSION_METHOD} pooling) MLP training for Coronal slices {TARGET_CORONAL_INDICES}...")
    print(f"Target Slice Identifiers for Fusion: {TARGET_SLICE_IDENTIFIERS}")
    print(f"Fused Feature Dim: {FEATURE_DIM_PER_SLICE}")
    print(f"Using device: {DEVICE}")
    print(f"Feature Root: {FEATURE_ROOT}")
    print(f"CSV Path: {CSV_PATH}")
    print(f"Best model will be saved to: {MODEL_SAVE_PATH}")
    print(f"Hyperparameters: Epochs={EPOCHS}, LR={LEARNING_RATE}, Batch={BATCH_SIZE}, Dropout={DROPOUT_RATE}, ES_Patience={EARLY_STOPPING_PATIENCE}")

    # --- Setup Datasets and DataLoaders ---
    try:
        print("Setting up datasets...")
        train_dataset = FusedSliceDataset(FEATURE_ROOT, CSV_PATH, 'train', TARGET_SLICE_IDENTIFIERS)
        val_dataset = FusedSliceDataset(FEATURE_ROOT, CSV_PATH, 'validation', TARGET_SLICE_IDENTIFIERS)
        test_dataset = FusedSliceDataset(FEATURE_ROOT, CSV_PATH, 'test', TARGET_SLICE_IDENTIFIERS)

        print("Setting up dataloaders...")
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    except (FileNotFoundError, ValueError, RuntimeError) as e:
        print(f"Error initializing datasets/loaders: {e}")
        raise SystemExit("Dataset/Loader initialization failed.")

    # --- Setup Model, Loss, Optimizer, Scheduler ---
    print("Initializing model, optimizer, scheduler...")
    model = FusedAgeMLP(input_dim=FEATURE_DIM_PER_SLICE, dropout_rate=DROPOUT_RATE).to(DEVICE)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler_patience_epochs = max(1, int(EPOCHS * SCHEDULER_PATIENCE_PERCENT))
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=scheduler_patience_epochs, factor=0.5, verbose=True)

    # --- Training Loop ---
    best_val_mae = float('inf')
    epochs_no_improve = 0
    best_epoch = -1
    history = {'train_loss': [], 'val_loss': [], 'val_mae': []}

    print(f"\n--- Starting Training ---")
    for epoch in range(EPOCHS):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
        val_loss, val_mae = evaluate(model, val_loader, criterion, DEVICE)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_mae'].append(val_mae)

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.3f}")

        scheduler.step(val_loss)

        if val_mae < best_val_mae:
            best_val_mae = val_mae
            best_epoch = epoch + 1
            epochs_no_improve = 0
            try:
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"  -> New best Val MAE: {best_val_mae:.3f}. Saved model to {MODEL_SAVE_PATH}")
            except Exception as e:
                print(f"  -> Error saving model: {e}")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
            print(f"\nEarly stopping triggered after {EARLY_STOPPING_PATIENCE} epochs without improvement.")
            break

        if (epoch + 1) % 20 == 0:
             gc.collect()
             if DEVICE.startswith('cuda'): torch.cuda.empty_cache()

    print(f"\n--- Training Finished ---")
    if best_epoch != -1:
         print(f"Best Validation MAE: {best_val_mae:.3f} achieved at epoch {best_epoch}")
         print(f"Best model saved to {MODEL_SAVE_PATH}")
    else:
         print("No improvement found during training or model saving failed.")

    # --- Evaluate on Test Set ---
    if MODEL_SAVE_PATH.exists() and best_epoch != -1:
        print(f"\n--- Evaluating on Test Set using Best Model ---")
        try:
            model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
            print(f"Loaded best model weights from {MODEL_SAVE_PATH}")
            test_loss, test_mae = evaluate(model, test_loader, criterion, DEVICE)
            print(f"Test Loss: {test_loss:.4f} | Test MAE: {test_mae:.3f}")
        except Exception as e:
            print(f"Error during test set evaluation: {e}")
    elif not MODEL_SAVE_PATH.exists():
         print(f"\nSkipping test set evaluation: Best model file not found at {MODEL_SAVE_PATH}")
    else:
         print("\nSkipping test set evaluation: No best model was saved during training.")

    print(f"\nFeature fusion ({FUSION_METHOD} pooling) MLP training script for Coronal {TARGET_CORONAL_INDICES} finished.")

Starting feature fusion (average pooling) MLP training for Coronal slices [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]...
Target Slice Identifiers for Fusion: ['120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130']
Fused Feature Dim: 256
Using device: cuda:1
Feature Root: /data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130
CSV Path: /data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv
Best model will be saved to: fused_mlp_model_avg_pool_coronal_120_130.pth
Hyperparameters: Epochs=500, LR=0.0001, Batch=32, Dropout=0.3, ES_Patience=40
Setting up datasets...

[Dataset Init] Split: train, Fusing Coronal Slices: ['120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130']
Loading features from base: /data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed_coronal_120_130
Loading metadata from: /data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv
Loaded metadata for 2850 subjects from 

Epoch 1/500 | Train Loss: 54.2875 | Val Loss: 53.6119 | Val MAE: 53.612
  -> New best Val MAE: 53.612. Saved model to fused_mlp_model_avg_pool_coronal_120_130.pth


Epoch 2/500 | Train Loss: 54.1740 | Val Loss: 53.4688 | Val MAE: 53.469
  -> New best Val MAE: 53.469. Saved model to fused_mlp_model_avg_pool_coronal_120_130.pth


Epoch 3/500 | Train Loss: 54.0290 | Val Loss: 53.4572 | Val MAE: 53.457
  -> New best Val MAE: 53.457. Saved model to fused_mlp_model_avg_pool_coronal_120_130.pth


Epoch 4/500 | Train Loss: 53.9281 | Val Loss: 53.3494 | Val MAE: 53.349
  -> New best Val MAE: 53.349. Saved model to fused_mlp_model_avg_pool_coronal_120_130.pth


...

Epoch 229/500 | Train Loss: 8.8160 | Val Loss: 7.0914 | Val MAE: 7.091


Epoch 230/500 | Train Loss: 9.0500 | Val Loss: 7.3193 | Val MAE: 7.319


Epoch 231/500 | Train Loss: 9.1384 | Val Loss: 6.9956 | Val MAE: 6.996


Epoch 232/500 | Train Loss: 9.0564 | Val Loss: 7.0950 | Val MAE: 7.095

Early stopping triggered after 40 epochs without improvement.

--- Training Finished ---
Best Validation MAE: 6.692 achieved at epoch 192
Best model saved to fused_mlp_model_avg_pool_coronal_120_130.pth

--- Evaluating on Test Set using Best Model ---
Loaded best model weights from fused_mlp_model_avg_pool_coronal_120_130.pth


Test Loss: 6.9427 | Test MAE: 6.943

Feature fusion (average pooling) MLP training script for Coronal [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130] finished.
